In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from environs import Env
env = Env()
env.read_env()
api_key=env("BINANCE_APIKEY")
api_secret=env("BINANCE_APISECRET")

In [52]:
from binance.client import Client
client = Client(api_key, api_secret)

from datetime import datetime
from func.utils import get_symbols, get_klines_df
from func.indicators import ST
from binance.enums import *
import numpy as np
import pandas as pd

In [4]:
def enrichDF(symbol, timeframe, limit=200):
    df = get_klines_df(symbol, timeframe, limit)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['open'] = df['open'].astype(float)
    df['close'] = df['close'].astype(float)
    df = df.drop(['close_time','taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)
    df = ST(df,ATR_factor,period)    # calculate SuperTrend
    df = df.drop(['H-L','H-PC','L-PC','TR','Upper Basic','Lower Basic'], axis=1)
    return df[:-1]    # exclude most recent bar

In [30]:
KLINE_INTERVAL = KLINE_INTERVAL_1HOUR
ATR_factor = 5
period = 10

symbol_data_db = dict()    # dictionary containing dataframes
symbol_list = ['BTCUSDT','ETHUSDT','XRPUSDT','BCHUSDT','EOSUSDT','LTCUSDT','TRXUSDT','ETCUSDT','LINKUSDT','XLMUSDT','ADAUSDT','BNBUSDT',
               'XMRUSDT','XTZUSDT','DASHUSDT','ZECUSDT','ONTUSDT', 'NEOUSDT','QTUMUSDT','IOTAUSDT']
signal_db = dict()

In [72]:
for symbol in symbol_list:
    symbol_data_db[symbol] = enrichDF(symbol, KLINE_INTERVAL_1HOUR)

In [73]:
buys = 0
sells = 0

for symbol in symbol_data_db:
    superTrend = symbol_data_db[symbol]['SuperTrend'].values[-1]
    upperBand = symbol_data_db[symbol]['Upper Band'].values[-1]
    changedDirection = symbol_data_db[symbol]['changedDirection'].values[-1]
    if superTrend == upperBand:
        signal = 'SELL'
        sells += 1
    else:
        signal = 'BUY'
        buys += 1
    if changedDirection:        
        print(symbol, 'Direction changed to', signal)
    else:
        print(symbol, signal)
print('Buys {}  Sells {}'.format(buys, sells))

BTCUSDT BUY
ETHUSDT BUY
XRPUSDT BUY
BCHUSDT BUY
EOSUSDT BUY
LTCUSDT BUY
TRXUSDT BUY
ETCUSDT SELL
LINKUSDT BUY
XLMUSDT BUY
ADAUSDT BUY
BNBUSDT BUY
XMRUSDT BUY
XTZUSDT BUY
DASHUSDT BUY
ZECUSDT BUY
ONTUSDT BUY
NEOUSDT BUY
QTUMUSDT BUY
IOTAUSDT BUY
Buys 19  Sells 1


In [52]:
symbol_data_db['ADAUSDT'].tail(10)

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,ATR,Upper Band,Lower Band,SuperTrend,changedDirection
189,1580871600000,0.05597,0.05610,0.05568,0.05602,5435253.20000000,303820.37807800,993,0.000618,0.058836,0.054008,0.054008,False
190,1580875200000,0.05604,0.05657,0.05597,0.05655,8911936.80000000,501608.53191900,1728,0.000616,0.058836,0.054008,0.054008,False
191,1580878800000,0.05655,0.05725,0.05654,0.05698,19011718.00000000,1083399.80720200,3392,0.000625,0.058836,0.054008,0.054008,False
192,1580882400000,0.05699,0.05775,0.05697,0.05727,16045241.40000000,921731.22919700,2694,0.000641,0.058836,0.054156,0.054156,False
193,1580886000000,0.05726,0.05758,0.05702,0.05715,8780858.00000000,503705.25297100,1616,0.000633,0.058836,0.054156,0.054156,False
194,1580889600000,0.05715,0.05722,0.05677,0.05707,7223601.70000000,411410.36177900,1302,0.000614,0.058836,0.054156,0.054156,False
195,1580893200000,0.05707,0.05768,0.05707,0.05748,10845702.50000000,623314.78138000,1980,0.000614,0.058836,0.054305,0.054305,False
196,1580896800000,0.05751,0.05811,0.05734,0.05763,21893713.00000000,1264131.47572800,3761,0.000630,0.058836,0.054577,0.054577,False
197,1580900400000,0.05766,0.05870,0.05756,0.05868,24770215.40000000,1439807.13195900,3215,0.000681,0.058836,0.054727,0.054727,False
198,1580904000000,0.05868,0.05955,0.05831,0.05835,39640182.80000000,2339914.90588300,6990,0.000737,0.058836,0.055247,0.055247,False


In [37]:
data_df = enrichDF('DASHUSDT', KLINE_INTERVAL_1HOUR)

In [45]:
data_df = data_df.rename(columns={"open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"})

In [46]:
data_df.tail(15)

,open_time,Open,High,Low,Close,Volume,quote_asset_volume,number_of_trades,ATR,Upper Band,Lower Band,SuperTrend,changedDirection
184,1581267600000,127.31,128.00,126.49,126.71,2677.35367000,340599.89105180,1406,1.955956,135.356080,119.285435,119.285435,False
185,1581271200000,126.71,127.46,126.69,127.16,1563.88717000,198811.27145620,1170,1.837360,135.356080,119.285435,119.285435,False
186,1581274800000,127.12,128.01,126.87,126.94,1814.65776000,231321.16939400,907,1.767624,135.356080,119.285435,119.285435,False
187,1581278400000,126.93,127.48,126.54,126.98,916.06950000,116397.62513540,659,1.684862,135.356080,119.285435,119.285435,False
188,1581282000000,126.96,127.56,126.80,126.94,1264.06771000,160739.83333660,663,1.592376,135.141878,119.285435,119.285435,False
189,1581285600000,126.91,128.00,126.81,127.32,2507.57133000,319785.60370720,993,1.552138,135.141878,119.644310,119.644310,False
190,1581289200000,127.34,128.66,127.05,128.51,4950.44368000,633637.74353110,1804,1.557924,135.141878,120.065379,120.065379,False
191,1581292800000,128.58,128.73,127.21,127.94,3574.82375000,457991.25687570,1602,1.554132,135.141878,120.199341,120.199341,False
192,1581296400000,127.94,128.40,127.21,127.44,2460.54813000,314443.07324090,1088,1.517719,135.141878,120.216407,120.216407,False
193,1581300000000,127.49,127.81,126.09,127.09,4628.69258000,587337.26961370,1759,1.537947,134.639734,120.216407,120.216407,False


In [29]:
from backtesting.lib import crossover

In [28]:
test_df = get_klines_df('BTCUSDT', KLINE_INTERVAL_1HOUR, 100)
test_df['open_time'] = pd.to_datetime(test_df['open_time'], unit='ms')

In [35]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
2004-08-26,104.95,107.95,104.66,107.91,3551000
2004-08-27,108.10,108.62,105.69,106.15,3109000
2004-08-30,105.28,105.49,102.01,102.01,2601000
2004-08-31,102.30,103.71,102.16,102.37,2461400
2004-09-01,102.70,102.97,99.67,100.25,4573700


In [47]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 10)
        self.ma2 = self.I(SMA, Close, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(data_df, SmaCross,
              cash=10000, commission=.002)
bt.run()
bt.plot()

/home/chekitsch/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Data index is not datetime. Assuming simple periods.
